In [1]:
%useLatestDescriptors
%use koog(v = 0.5.0)

In [2]:
import kotlinx.coroutines.runBlocking

val apiKey = System.getenv("OPENAI_API_KEY")
val executor = simpleOpenAIExecutor(apiKey)

runBlocking {
    val res = executor.execute(
        prompt = prompt("prompt") {
            user("hello")
        },
        model = OpenAIModels.Chat.GPT5Mini
    )
    res.last().content
}


Hi — how can I help you today? (I can answer questions, write or edit text, help with code, summarize or translate, brainstorm, and more.)

In [3]:
import ai.koog.agents.features.eventHandler.feature.handleEvents
import ai.koog.agents.features.opentelemetry.feature.OpenTelemetry
import ai.koog.agents.features.opentelemetry.integration.langfuse.addLangfuseExporter
import org.intellij.lang.annotations.Language

val toolRegistry = ToolRegistry {
    tool(AskUser)
}

runBlocking {
    val agent = AIAgent(
        promptExecutor = executor,
        llmModel = OpenAIModels.Chat.GPT5Mini,
        systemPrompt = """
        あなたは旅行プランナーです。ユーザーの指示に従って、旅行計画を立ててください。
        ただしユーザーの指示が少ないときは__ask_user__ツールを使って、1度はユーザーに情報提供を促してください。
        最小限の情報があればそれでよいです。観光スポットなども含めて、あなたが考えて提案してください。
        ユーザーに行きたい場所を尋ねず、あなたが提案してください。
        最小限の情報とは、旅行先、予算のことです。
        あまり細かく聞きすぎず、ある程度分かったところで計画を立ててください
        """.trimIndent(),
        toolRegistry = toolRegistry,
    ) {
        install(OpenTelemetry) {
            setVerbose(true)
            addLangfuseExporter()
        }
        handleEvents {
            onToolCallCompleted {
                when (it.tool) {
                    is AskUser -> {
                        println("---")
                        println("user message: ${it.result}")
                    }

                    else -> {}
                }
            }
        }
    }

    val res = agent.run("2泊3日の札幌旅行の計画を立ててください")
    println("---")
    println("Assistant response:")
    println(res)
}

ご予算を教えてください（お一人当たりの目安で、通貨は日本円でお願いします）。例：3万円、5万円、10万円など。
---
user message: 10月上旬、予算一人5万。二人で行きます。シマエナガをみたいです。
---
Assistant response:
ありがとうございます。いただいた情報（10月上旬、2名、予算お一人5万円、シマエナガを見たい）で、2泊3日の札幌出発（札幌市内滞在中心）プランを作成します。1点確認させてください：お一人5万円のご予算は「ご自宅→札幌の往復（飛行機・新幹線等）を含む総額」ですか、それとも「札幌到着後の現地滞在費（宿泊・食事・現地移動・ツアー等）のみ」ですか？（どちらでも対応しますので教えてください）

以下は、確認前提として「到着は新千歳空港、予算は札幌滞在中を中心に使う／往復交通は別途扱い」と想定した標準プランです。航空券を含めたい場合は回答で教えてください（調整します）。

2泊3日サンプルプラン（シマエナガ観察を中心）

Day1：到着日（札幌市内観光）
- 午前〜午後：新千歳空港到着 → JR快速エアポートで札幌へ（所要約37分）
- チェックインは札幌中心部（大通・すすきの）で便利なホテルを推奨（例：札幌グランドホテル、メルキュール、京王プレリアなど：中級〜やや上の快適クラス）
- 夕方：大通公園、さっぽろテレビ塔、狸小路・すすきので夕食（札幌味噌ラーメン、海鮮、スープカレー等）
- 所要時間：自由。疲れがあれば早めに休息。

Day2：シマエナガ観察の日（早朝発→夕方帰着）
- 早朝発（5:00〜6:00出発が理想）
- オプションA（推奨：日帰りで本気で探す）：
  - JR＋車/ガイドで旭川〜大雪山（大雪山麓、裾合平などの白樺・ダケカンバ林）へ。シマエナガは山地の樹林帯で見られることが多く、早朝がチャンス。
  - 専門のバードウォッチングガイド（半日〜終日）を事前予約すると成功率が高まります（地元ガイドが生息域・ポイントを熟知）。
  - 午後に旭川で軽く観光（旭川ラーメン等）して札幌へ戻る。
- オプションB（移動少なめ・楽に）：
  - 札幌近郊（円山公園・手稲・定山渓周辺の林や郊外フィールド）で早朝バードウォッチング。地元のバードツアー参加を推奨。
- 夕方：札幌で温泉や地元グルメ（海鮮、炉端、ジンギ